In [ ]:
MySQL Physical Cold Backup
=============================================

-- Cold backup involves physically copying database related files to backup device
   taking the database down.
-- for this the mysql database server should be clean shutdown.

files required in cold backup
==========================================

-- all the files in datadir.
-- all the files of system tablespaces.
-- all the configuration/option files.

files not required in cold backup
============================================

-- redo log files , undo log files , bin log files 
-- temp files , double write buffer files.


clean shutdown the database
-----------------------------------------

mysql> set global innodb_fast_shutdown = 0;

[root@mysqlserver mysql]# systemctl stop mysqld


copy files to backup location 
------------------------------------------

[root@mysqlserver mysql]# cp -r /var/lib/mysql/data /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -r /var/lib/mysql/systemdata /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /etc/mysql/my.cnf /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /var/lib/mysql-files/*.cnf /tmp/mysql_cold_backup/


to restore mysql instance from the cold backup , remove all the files 
-------------------------------------------------------------------------

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*


copy the files from backup
-------------------------------------

[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/data/* /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/systemdata/* /var/lib/mysql/systemdata/

[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/systemdata/


start the mysql instance 
-------------------------------------

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 

In [ ]:
Logical Backups
==============================

-- does not involve copying of physical files.
-- all databases , some databases and table level backup can be taken.
-- utility have to be used like mysqldump amd mysqlpump.
-- takes backup in .sql format , text delimited or XML format.

[mysqluser@mysqlserver backups]$ mysqldump --verbose --help

Dumping structure and contents of MySQL databases and tables.
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases [OPTIONS] DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases [OPTIONS]


specifying defaults for mysqldump 
--------------------------------------------

[root@MySqlServer1 backups]# cat /etc/my.cnf
[mysqldump]
host=localhost
user=db_admin
password=Dbadmin@2024

[root@MySqlServer1 backups]# mysqldump --print-defaults

mysqldump would have been started with the following arguments:
--host=localhost --user=db_admin --password=***** 
[root@MySqlServer1 backups]# 


Single Table Backup
-----------------------

mysqldump sakila actor > sakilaDB_actorTable_bkp.sql


Whole Single Database Backup 
-------------------------------

mysqldump sakila > sakilaDB_bkp.sql


Two databases backup
--------------------------

mysqldump --databases posdb sakila > posdb_sakila_bkp.sql


All databases backup 
-----------------------------

mysqldump --all-databases > all_dbs_bkp.sql


Database backup with skipping some tables
-------------------------------------------------

mysqldump sakila --ignore-table=sakila.actor --ignore-table=sakila.film_actor > 
          sakiladb_skipactor_bkp.sql


Multiple database backup with skipping some tables
-------------------------------------------------------

mysqldump --databases sakila posdb --ignore-table=sakila.actor --ignore-table=sakila.film_actor 
          --ignore-table=posdb.emp_salaries > sakiladb_posdb_ignoretables_bkp.sql


consistent backup 
-------------------------------------------------

mysqldump --single-transaction > all_dbs_consistent_bkp.sql


consistent backup without locking tables
-------------------------------------------------

mysqldump --all-databases --single-transaction --skip-lock-tables > all_dbs_consistent_bkp_dump.sql


definition backup without data 
--------------------------------------------

mysqldump -u root -p --no-data sakila > /home/mysqluser/backups/sakila_db_backup_definition.sql


only data backup without definition ( create statements )
------------------------------------------------------------------

mysqldump -u root -p --no-create-info sakila > /home/mysqluser/backups/sakila_db_backup_dataonly.sql


backup databases with compressed backup file 
------------------------------------------------------

mysqldump -u root -p --databases sample sakila | gzip > dump_db_sample_sakila.sql.gz


Dump large tables with quick option 
-----------------------------------------------

mysqldump --comments --quick airportdb > airportdb_dump_quick.sql


copy data from one host to another using mysqldump
-------------------------------------------------------

mysqldump ordersdb | mysql -h 192.168.153.137 -u db_admin -p******** orders


Dump a remote database
--------------------------------------------

mysqldump -h bosprod01 --port 3306 -u db_admin -p sakila > dump_bosprod01_sakiladb.sql


Use where clause to filter data
-----------------------------------------

mysqldump sample sample_employees --where="employee_id > 1000" > dump_sample_employees_custid1000.sql


Limit backup file using where clause
------------------------------------------------

mysqldump sample --where='1 limit 500' > dump_sample_500records.sql


Dump database with routines ( by default mysqldump does not include routines)
---------------------------------------------------------------------------------

mysqldump Northwind --routines > dump_northwind.sql




Single Table Restore from backup taken for single table
---------------------------------------------------------------

mysql> drop table actor;

mysql -u db_admin -p sakila < sakilaDB_actorTable_bkp.sql


Single Table restore from backup taken for full database
--------------------------------------------------------------

ALL CREATE TABLE statements start with DROP TABLE IF EXISTS and then CREATE TABLE and then
INSERT STATEMENTS and it will end with keyword UNLOCK TABLES

sed -n -e '/DROP TABLE.*`film_actor`/,/UNLOCK TABLES/p' sakilaDB_bkp.sql > sakilaDB_filmactor_bkp.sql

drop table actor ;

mysql sakila < sakilaDB_filmactor_bkp.sql


Database restore from backup taken for a single database
---------------------------------------------------------------------

mysql> drop database sakila;

mysql sakila < sakilaDB_bkp.sql


Restore a single database from all database backup
-------------------------------------------------------

cat dump_alldatabases.sql | grep 'CREATE DATABASE' | grep -v mysql

mysql> drop database ordersdb;

mysql> drop database sakila;

If the create database is in between the dump 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`ordersdb`/,/CREATE DATABASE/p' dump_alldatabases.sql > ordersdb_bkp.sql

remove the last create datbase statement in ordersdb_bkp.sql

mysql < ordersdb_bkp.sql

If the create database is the last 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`sakila`/,/Dump completed/p' dump_alldatabases.sql > sakiladb_bkp.sql

mysql < sakiladb_bkp.sql

In [ ]:
MySql Hot Backup with Xtrabackup 
====================================================

-- Hot backup involves physically copying database related files to backup device
   when the database is Up.

-- Tools - mysqlbackup ( oracle , paid) , mariadbbackup ( mariadb , open-source)
           xtrabackup ( percona , open-source )

-- point-in-time recovery can be performed using hot backup and binary logs.


download xtrabackup utility
----------------------------------------------

https://www.percona.com/downloads#percona-xtrabackup

yum localinstall percona-xtrabackup-80-8.0.30-23.1.el8.x86_64.rpm 

xtrabackup --version

create user for taking backups and grant permissions
---------------------------------------------------------

mysql> create user 'bkpuser'@'%' identified by 'Bkpuser@2024';

mysql> grant reload , lock tables , process , replication client on *.* to 'bkpuser'@'%' ;

mysql> grant backup_admin on *.* to 'bkpuser'@'%' ;

mysql> grant select on performance_schema.log_status to 'bkpuser'@'%' ;

mysql> grant select on performance_schema.keyring_component_status to 'bkpuser'@'%' ;

mysql> flush privileges;


Take full backup of mysql instance with xtrabackup 
---------------------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups

xtrabackup --prepare --target-dir=/home/mysqluser/backups


Restore using the backup taken through xtrabackup utility
--------------------------------------------------------------

systemctl stop mysqld

rm -rf /var/lib/mysql/*  ( remove all files except binlog and errorlog )

xtrabackup --copy-back --target-dir=/home/mysqluser/backups/

chown -R mysql:mysql /var/lib/mysql/

systemctl start mysqld




xtrabackup incremental backups
-------------------------------------------

create a full base backup of the database
-------------------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/base

create an incremental-1 database backup 
-------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/incr1 
           --incremental-basedir=/home/mysqluser/backups/base

create an incremental-2 database backup
----------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/incr2 
           --incremental-basedir=/home/mysqluser/backups/incr1


prepare the full and incremental backups for restoration
----------------------------------------------------------------

Need to add --apply-log-only option when preparing incremental backups except for the last one.
This is to prevent the rollback phase , else the incremental backups becomes useless.

xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base

xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base 
           --incremental-dir=/home/mysqluser/backups/incr1

xtrabackup --prepare --target-dir=/home/mysqluser/backups/base 
           --incremental-dir=/home/mysqluser/backups/incr2




Point-In-Time Recovery using Xtrabackup and binary logs
----------------------------------------------------------------

run the full backup 
----------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups

xtrabackup --prepare --target-dir=/home/mysqluser/backups

prepare for restoration by emptying all the mysql data , systemdata , redolog directories
---------------------------------------------------------------------------------------------

Don't remove files in errorlog and binlog directories.

systemctl stop mysqld

[root@mysqlserver backups]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/systemdata/*


move binlogs to different location , if there is backup of binlog you can remove this
----------------------------------------------------------------------------------------

mv posb_binlog.* ../old_binlog/


restore all the files from the full backup and verify
--------------------------------------------------------------------------

xtrabackup --copy-back --target-dir=/home/mysqluser/backups

chown -R mysql:mysql /var/lib/mysql/


check the xtrabackup_binlog_info for log file name and position
---------------------------------------------------------------------------

[root@mysqlserver backups]# cat xtrabackup_binlog_info
posb_binlog.000010	493
[root@mysqlserver backups]# 


create a restore file with this info and the time to which the database has to be restored 
--------------------------------------------------------------------------------------------

mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000010 --start-position=493 
            --stop-datetime="2024-05-21 18:02:30" > restore_db.sql


start the mysqld server and run the restore_db.sql
----------------------------------------------------------

systemctl start mysqld

mysql < restore_db.sql



xtrabackup compressed backup 
---------------------------------------

xtrabackup --backup --compress --target-dir=/home/mysqluser/backups/full_compressed


compressed backups should be decompressed before prepare and restore
------------------------------------------------------------------------

xtrabackup --decompress --target-dir=/home/mysqluser/backups/full_compressed



Creating partial backups
---------------------------------------------

all tables in a database 
--------------------------------

xtrabackup --backup --target-dir=/home/backups/ordersdb_all_tables/ --tables="^ordersdb[.].*"


single table in a database 
----------------------------------

xtrabackup --backup --target-dir=/home/backups/ordersdb_products01_table/ 
           --tables="^ordersdb[.]products01"


table files option 
-----------------------

cat ordersdb_tables.txt
ordersdb.products01
ordersdb.products02

xtrabackup --backup --target-dir=/home/backups/ordersdb_tables/ 
           --tables-file=/home/mysqluser/backups/ordersdb_tables.txt


single database backup
--------------------------------------

xtrabackup --backup --databases='mysql sys performance_schema ordersdb' 
           --target-dir=/home/mysqluser/backups/ordersdb_bkp/


database files option
-----------------------------------------

cat databases_for_bkp.txt 
mysql
sys
performance_schema
salesdb

xtrabackup --backup --databases-file=/home/mysqluser/backups/databases_for_bkp.txt
           --target-dir=/home/mysqluser/backups/databases_bkp/


preparing partial backups and restore 
----------------------------------------------------------

xtrabackup --prepare --export --target-dir=/home/mysqluser/backups/ordersdb_all_tables

if table is present and you want to restore 
---------------------------------------------

mysql> alter table ordersdb.products01 discard tablespace;

cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products01.*  /var/lib/mysql/data/ordersdb/

chown mysql:mysql /var/lib/mysql/data/ordersdb/products01.*

mysql> alter table ordersdb.products01 import tablespace;

if table is dropped and you want to restore
---------------------------------------------------

CREATE TABLE `products02` (
  `prod_id` int NOT NULL,
  `prod_details` varchar(100) DEFAULT NULL,
  `prod_date` datetime DEFAULT NULL,
  PRIMARY KEY (`prod_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

alter table products02 discard tablespace;

cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products02.* 
   /var/lib/mysql/data/ordersdb/

chown mysql:mysql /var/lib/mysql/data/ordersdb/products02.*

alter table products02 import tablespace;